## Evaluation

In [ ]:
import re, math, pathlib
from collections import Counter
import nltk
import pandas as pd
from nltk.util import ngrams
import math
from collections import Counter

# grammar/spell checker
try:
    import enchant
    IT_DICT = enchant.Dict("it_IT")          # Italian dictionary
except Exception:
    IT_DICT = None   # spell-error rate will be skipped if dict not present


# ----------  sentence & word tokenizers ----------
# 1) try native Italian; 2) English Punkt; 3) split
def _build_sentence_splitter():
    try:                                   
        return nltk.data.load('tokenizers/punkt/italian.pickle').tokenize
    except (LookupError, OSError):
        try:                                
            nltk.download('punkt', quiet=True)
            return nltk.data.load('tokenizers/punkt/english.pickle').tokenize
        except Exception:                   
            return lambda txt: re.split(r'(?<=[.!?])\s+', txt.strip())

sent_split = _build_sentence_splitter()
word_tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")

def sentences(text):
    return [s for s in sent_split(text) if s.strip()]

def words(text):
    return word_tokenizer.tokenize(text)

# ----------  syllables (for Gulpease / F-V) ----------
vowel_re = re.compile(r'[aeiouyàèéìòóù]+', re.I)
def syllables(word:str):
    return len(vowel_re.findall(word))

# ----------  readability ----------
def gulpease(text):
    w = words(text); s = sentences(text)
    if not w or not s: return None
    letters = sum(len(wd) for wd in w)
    return 89 + (300*len(s) - 10*letters) / len(w)

def flesch_vacca(text):
    w = words(text); s = sentences(text); syll = sum(syllables(wd) for wd in w)
    if not w or not s: return None
    return 206 - 1.3*(len(w)/len(s)) - 60*(syll/len(w))

# ----------  writing quality ----------
def spell_error_rate(text):
    if IT_DICT is None: return None
    tokens = [t for t in words(text) if t.isalpha()]
    if not tokens: return None
    errors = sum(not IT_DICT.check(t) for t in tokens)
    return errors / len(tokens)

def avg_sentence_length(text):
    s = sentences(text); w = words(text)
    return len(w)/len(s) if s else None

# ----------  heuristic named‑entity extraction (no spaCy) ----------
_caps = re.compile(r'\b[A-Z][A-Za-zÀ-ÖØ-Ýà-öø-ý]+\b')  # capitalised words

def _heuristic_entities(txt:str):
    ents = []
    for sent in sentences(txt):
        for tok in words(sent):
            if _caps.match(tok) and not tok.isupper():        # skip acronyms 
                ents.append(tok)
    # also keep 4‑digit numbers (years, model names, etc.)
    ents.extend(re.findall(r'\b\d{4}\b', txt))
    return set(ents)

def entity_preservation_ratio(orig_en:str, summary_it:str):
    ent_en = _heuristic_entities(orig_en)
    ent_it = _heuristic_entities(summary_it)
    return len(ent_en & ent_it) / len(ent_en) if ent_en else None



In [3]:
orig_dir = pathlib.Path("originals")
sum_dir  = pathlib.Path("summaries")

records = []

for name in ["RL.txt", "DL.txt", "IR.txt", "ML.txt"]:
    en = (orig_dir / name).read_text(encoding='utf-8')
    it = (sum_dir  / name).read_text(encoding='utf-8')

    rec = {
        "file":          name,
        # readability
        "gulpease":      round(gulpease(it), 1),
        "flesch_vacca":  round(flesch_vacca(it), 1),
        # writing quality
        "spell_err":     round(spell_error_rate(it), 3) if spell_error_rate(it) is not None else "—",
        "avg_sent_len":  round(avg_sentence_length(it), 1),

    }
    records.append(rec)

df = pd.DataFrame(records)
print(df.to_string(index=False))


  file  gulpease  flesch_vacca spell_err  avg_sent_len
RL.txt      43.9          32.0         —          19.8
DL.txt      29.7          -7.8         —          42.3
IR.txt      48.7          40.6         —          17.8
ML.txt      45.7          31.1         —          16.1
